In [ ]:
##  计算每种菌在每种疾病中的信号差异
##  包括 1. 丰度层面； 2. 存在率层面  3. 多样性层面【alpha 多样性指数，effect size】 4. 总丰度

In [14]:
import pandas as pd
import numpy as np
# from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
from scipy.stats import wilcoxon
from scipy.stats import mannwhitneyu

In [5]:
# 导入数据
df = pd.read_csv('02_species_abundance.id_92', sep='\t', index_col = 0)

# 去除第一行unmapped
df = df.iloc[1:]

# Abundnace table
df_abundance = df.filter(regex='Relative Abundance \(%\)')

# 删除样本名中携带的后缀
df_abundance.columns = df_abundance.columns.str.replace('.sort Relative Abundance (%)', '')

# RPKM table#
df_rpkm = df.filter(regex='RPKM')
# 将'.sort RPKM'替换为""
df_rpkm.columns = df_rpkm.columns.str.replace('.sort RPKM', '')
# 将'.filt.sort.RPKM'替换为""
df_rpkm.columns = df_rpkm.columns.str.replace('.filt RPKM', '')


# Coverage table
df_coverage = df.filter(regex='Reads\ per\ base')
# 删除样本名中携带的后缀
df_coverage.columns = df_coverage.columns.str.replace('.sort Reads per base', '')

## 读取Mapping 文件
mp = pd.read_csv('mapping.txt.f1', sep='\t')

# 使用未标准化的RPKM数据
dt_rpkm_input = df_rpkm.T
## 为样本加上Label
dt_rpkm_input_mp = pd.merge(mp, dt_rpkm_input, left_on='Sample', right_index=True)

In [6]:
## 筛选以iLABdb开头的列名
iLABdb_cols = [col for col in dt_rpkm_input_mp.columns if col.startswith('iLABdb')]
dt_mt = pd.melt(dt_rpkm_input_mp, id_vars=["Sample", "BioProject", "Type", "Group", "LevelB", "LevelB1", "Project", "LevelA2", "LevelA3"], value_vars=iLABdb_cols, var_name='Variable', value_name='Value')

In [15]:
# 计算P值和LogFC
results = []
for (level, species), group_df in dt_mt.groupby(['LevelB1', 'Variable']):
    control_values = group_df.loc[group_df['Group'] == 'Control', 'Value']
    disease_values = group_df.loc[group_df['Group'] == 'Disease', 'Value']
    
    # # t-test
    # t_stat, p_value = ttest_ind(control_values, disease_values, equal_var=False, nan_policy='omit')
    stat, p_value = mannwhitneyu(control_values, disease_values, alternative='two-sided')
    
    # LogFC
    log_fc = np.log2(np.mean(disease_values) / np.mean(control_values))
    
    results.append({'LevelB1': level, 'Variable': species, 'PValue': p_value, 'LogFC': log_fc})

results_df = pd.DataFrame(results)

# BH校正
adjusted_pvalues = multipletests(results_df['PValue'], method='fdr_bh')[1]
results_df['AdjustedPValue'] = adjusted_pvalues


In [17]:
## 分别统计LogFC列<0 且 AdjustedPValue<0.05； 以及LogFC列>0 且 AdjustedPValue<0.05的列数量
## LogFC值<0代表在在疾病中缺少
# 统计LogFC < 0 且 AdjustedPValue < 0.05的行数
count_negative_logfc_and_significant = results_df[(results_df['LogFC'] < 0) & (results_df['AdjustedPValue'] < 0.05)].shape[0]

# 统计LogFC > 0 且 AdjustedPValue < 0.05的行数
count_positive_logfc_and_significant = results_df[(results_df['LogFC'] > 0) & (results_df['AdjustedPValue'] < 0.05)].shape[0]

print(f"满足LogFC < 0 且 AdjustedPValue < 0.01的行数: {count_negative_logfc_and_significant}")
print(f"满足LogFC > 0 且 AdjustedPValue < 0.01的行数: {count_positive_logfc_and_significant}")

满足LogFC < 0 且 AdjustedPValue < 0.01的行数: 1093
满足LogFC > 0 且 AdjustedPValue < 0.01的行数: 624


In [ ]:
## 考虑细菌的Mapping coverage，再进行Marker的计算
